In [ ]:
#

# Check graphics card


#remeber to 'cd' into another folder in the terminal before running the conection stuff
#like 'cd D:\colab'
!nvidia-smi

Fri Mar 17 12:09:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 473.62       Driver Version: 473.62       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:2D:00.0  On |                  N/A |
| 77%   38C    P2    41W / 216W |   8080MiB /  8088MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#@title Clone github repo
!git clone https://github.com/effusiveperiscope/so-vits-svc -b eff-4.0

Cloning into 'so-vits-svc'...


In [ ]:
#@title Install dependencies

!pip install pyworld praat-parselmouth
!python -m pip install --upgrade pip
!pip install fairseq==0.12.2 librosa==0.8.1

  Using cached pyworld-0.3.2-cp310-cp310-win_amd64.whl (209 kB)
  Using cached praat_parselmouth-0.4.3-cp310-cp310-win_amd64.whl (8.9 MB)
  Using cached Cython-0.29.33-py2.py3-none-any.whl (987 kB)
  Using cached fairseq-0.12.2-cp310-cp310-win_amd64.whl
  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Using cached sacrebleu-2.3.1-py3-none-any.whl (118 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached regex-2022.10.31-cp310-cp310-win_amd64.whl (267 kB)
  Using cached omegaconf-2.0.6-py3-none-any.whl (36 kB)
  Using cached bitarray-2.7.3-cp310-cp310-win_amd64.whl (118 kB)
  Using cached hydra_core-1.0.7-py3-none-any.whl (123 kB)
  Using cached scikit_learn-1.2.2-cp310-cp310-win_amd64.whl (8.3 MB)
  Using cached soundfile-0.12.1-py2.py3-none-win_amd64.whl (1.0 MB)
  Using cached pooch-1.7.0-py3-none-any.whl (60 kB)
  Using cached audioread-3.0.0-py3-none-any.whl
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)
  Using cached numba-0.56.4-cp310-cp310-win_

In [ ]:
import os
os.chdir ('./so-vits-svc/')

os.getcwd()

'G:\\colab\\so-vits-svc'

In [ ]:
#@title Download ContentVec model file
!pip install wget
import wget

url = 'https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/checkpoint_best_legacy_500.pt'
filename = wget.download(url)
print(filename)

  Using cached wget-3.2-py3-none-any.whl
100% [................................................] 1330114945 / 1330114945checkpoint_best_legacy_500.pt


In [ ]:
#@title Backup link (in case the above do not work)
#anonfiles.com/ScObw8d0z6/checkpoint_best_legacy_500_pt
#^open this link in brose, right click on the download button and "copy the adress" to the 'url' portion below

!pip install wget
import wget
url = 'https://cdn-149.anonfiles.com/ScObw8d0z6/b89b1220-1678487735/checkpoint_best_legacy_500.pt'
filename = wget.download(url)
print(filename)

In [ ]:
#@title Move file to hubert folder

import shutil

shutil.move("checkpoint_best_legacy_500.pt", "./hubert")


'./hubert\\checkpoint_best_legacy_500.pt'

# Dataset preprocessing

for sake of simplicity, create a folder 'dataset' and place your zip file in there and fill out the rest of this info.

Alterantively, ignore this one cell, and place your stuff in "so-vits-svc/dataset_raw/(your_character_name)/wavs_files.wav" directory and run the resample cell.


In [ ]:
#@title Load the dataset from .zip in Google Drive for preprocessing
#@markdown Name of the zip folder
DATASETNAME = "NamelessHero_eng"  #@param {type:"string"}
#@markdown Zip path (usually do not need to change this unless you uploaded to a different directory)
ZIP_PATH = "./dataset/"  #@param {type:"string"}
ZIP_NAME = ZIP_PATH + DATASETNAME

!unzip -d /content/so-vits-svc/dataset_raw {ZIP_NAME}.zip

In [ ]:
#@title Resample to 44.1k
!python resample.py

./dataset_raw\TreeHugger



0it [00:00, ?it/s]
1it [00:15, 15.07s/it]
9it [00:15,  1.22s/it]
11it [00:15,  1.38s/it]


In [ ]:
#@title Segment training set and generate configuration files
!python preprocess_flist_config.py

Writing ./filelists/train.txt
Writing ./filelists/val.txt
Writing ./filelists/test.txt
Writing configs/config.json



100%|##########| 1/1 [00:00<00:00, 499.98it/s]

100%|##########| 7/7 [00:00<?, ?it/s]

100%|##########| 2/2 [00:00<?, ?it/s]

100%|##########| 2/2 [00:00<?, ?it/s]


In [ ]:
#@title Generate hubert and f0
!python preprocess_hubert_f0.py

[11]
Loading hubert for content...
load model(s) from hubert/checkpoint_best_legacy_500.pt
INFO:fairseq.tasks.text_to_speech:Please install tensorboardX: pip install tensorboardX
INFO:fairseq.tasks.hubert_pretraining:current directory is G:\colab\so-vits-svc
INFO:fairseq.tasks.hubert_pretraining:HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
INFO:fairseq.models.hubert.hubert:HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, 'layer_type': transformer, 'dropout': 0.1, 'attention_dropout': 0.1, 'activati


100%|##########| 11/11 [00:16<00:00,  1.50s/it]


# Training

In [ ]:
#@title  download the G_0 and D_0 models
#anonfiles.com/GaX4w6dfz7/G_0_pth
#anonfiles.com/m8Wew6d8z9/D_0_pth
#^open this link in browse, right click on the download button and "copy the adress" to the 'url' portion below


import wget
#url = 'https://cdn-149.anonfiles.com/GaX4w6dfz7/4c3a0170-1678490854/G_0.pth'
url = 'https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/G_0.pth'
filename = wget.download(url)
print(filename)
#url = 'https://cdn-153.anonfiles.com/m8Wew6d8z9/2ef7633b-1678490871/D_0.pth'
url = 'https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/D_0.pth'
filename = wget.download(url)
print(filename)

100% [..................................................] 180628517 / 180628517G_0.pth
100% [..................................................] 187018591 / 187018591D_0.pth


In [ ]:
#@title  move the G_0 and D_0 models

import shutil
shutil.move("G_0.pth", ".\\logs\\44k")
shutil.move("D_0.pth", ".\\logs\\44k")

'.\\logs\\44k\\D_0.pth'

In [ ]:
!pip install tensorboard
!pip install matplotlib

  Using cached tensorboard-2.12.0-py3-none-any.whl (5.6 MB)
  Using cached grpcio-1.51.3-cp310-cp310-win_amd64.whl (3.7 MB)
  Using cached protobuf-4.22.1-cp310-abi3-win_amd64.whl (420 kB)
  Using cached Werkzeug-2.2.3-py3-none-any.whl (233 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached google_auth-2.16.2-py2.py3-none-any.whl (177 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached Markdown-3.4.1-py3-none-any.whl (93 kB)
  Using cached tensorboard_data_server-0.7.0-py3-none-any.whl (2.4 kB)
  Using cached cachetools-5.3.0-py3-none-any.whl (9.3 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl (151 kB)
  Using cached m

In [ ]:
#run this cell and cell below
import os
os.getcwd()

'G:\\colab\\so-vits-svc'

In [ ]:
#@title python: can't open file 'G:\\colab\\train.py': [Errno 2] No such file or directory

import os
os.chdir ('./so-vits-svc/')

os.getcwd()

'G:\\colab\\so-vits-svc'

edit the so-vits-svc\configs\config.json file from batch 6 to batch 1 (as standard gaming gpus will not able to handle large batch trainings).

Sadly the code is broken and will not restart from last point on its own, for restarting training, go to the 'logs/44k/ folder and move or remove all the D_ and G_ models from directory with exception for the latest ones, and rename them to D_0 and G_0.
italicized text

Than run two cells above than run the cell below.

Recommeneded to train the model in one go.

In [ ]:
#@title  Start training
#@markdown **Start training**
Clone = "44k"

#@markdown **Enable tensorboard for data visualization (dont turn it on, tensorboard seems to be broken here)**
tensorboard_on = False #@param {type:"boolean"}
if tensorboard_on:
  %load_ext tensorboard
  %tensorboard --logdir logs\"{Clone}"

!python train.py -c configs/config.json -m "{Clone}"


^C
